In [1]:
import warnings
warnings.filterwarnings('ignore')

## 1. Bi-Encoder

In [2]:
from sentence_transformers import SentenceTransformer, models

word_embedding_model = models.Transformer('klue/roberta-base')
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [3]:
print(model)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)


In [4]:
import torch

In [5]:
def mean_pooling(model_output, attention_mask):
  token_embeddings = model_output[0]
  input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()

  sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
  sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)

  return sum_embeddings / sum_mask

In [6]:
def max_pooling(model_output, attention_mask):
  token_embeddings = model_output[0]
  input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()

  token_embeddings[input_mask_expanded == 0] = 1e-9

  return torch.max(token_embeddings, 1)[0]

## 2. Sentence-Transformers : text

In [7]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

embs = model.encode([
    '잠이 안 옵니다',
    '졸음이 옵니다',
    '기차가 옵니다'
])

cos_scores = util.cos_sim(embs, embs)

print(cos_scores)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/336k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/967k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

tensor([[1.0000, 0.6410, 0.1887],
        [0.6410, 1.0000, 0.2730],
        [0.1887, 0.2730, 1.0000]])


## 3. Sentence-Transformers : image

In [8]:
from PIL import Image
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('clip-ViT-B-32')

img_embs = model.encode([
    Image.open('11-week-dog.jpg'),
    Image.open('11-week-cat.jpg')
])

txt_embs = model.encode([
    'A dog on grass',
    'Brown cat on yellow background'
])

cos_scores = util.cos_sim(img_embs, txt_embs)

print(cos_scores)

modules.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.91k [00:00<?, ?B/s]

0_CLIPModel%2Fmerges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

0_CLIPModel%2Fconfig.json:   0%|          | 0.00/4.03k [00:00<?, ?B/s]

...

0_CLIPModel%2Fpreprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

0_CLIPModel%2Ftokenizer_config.json:   0%|          | 0.00/604 [00:00<?, ?B/s]

0_CLIPModel%2Fvocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

tensor([[0.2782, 0.1512],
        [0.2071, 0.3180]])
